In [1]:
from flask import Flask, render_template, request, session, redirect
from flask_session import Session
import pyodbc
import pandas as pd

In [2]:
dados_conexao = ("Driver={SQL Server};"
                "Server=INSIRA_AQUI_SEU_SERVIDOR;"
                "Database=poker;")
conexao = pyodbc.connect(dados_conexao)
cursor = conexao.cursor()

In [5]:
app = Flask(__name__)
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

In [6]:
@app.route("/")
def index():
    if not session.get("name"):
        return redirect("/login")
    return render_template("inicio.html")

In [7]:
@app.route("/login", methods = ["POST", "GET"])
def login():
    if request.method == "POST":
        session["name"] = request.form.get("name")
        return redirect("/")
    return render_template("login.html")

In [8]:
@app.route("/logout", methods = ["POST", "GET"])
def logout():
    session["name"] = None
    return redirect("/")

In [9]:
@app.route("/participantes", methods=["POST", "GET"])
def participantes():
    nome_p1 = session["name"]
    nro_participantes = int(request.form.get("nro_participantes"))
    quantia_inicial = int(request.form.get("quantia_inicial"))
    return render_template("participantes.html", nro_participantes=nro_participantes, quantia_inicial=quantia_inicial, nome_p1=nome_p1)

In [10]:
@app.route("/poker", methods=["POST","GET"])
def poker():
    nro_participantes = int(request.form.get("nro_part"))
    quantia_inicial = request.form.get("qt_inicial")
    part = []
    for i, item in enumerate(range(nro_participantes)):
        part.append(request.form.get("p"+str(i+1)))
    db.execute("DELETE FROM participantes")
    db.execute("DELETE FROM transacoes")
    for i, participante in enumerate(range(nro_participantes)):
        db.execute("INSERT INTO participantes(nome, quantia) VALUES(?, ?)", part[i], quantia_inicial)
    pote = "Pote"
    db.execute("INSERT INTO participantes(nome, quantia) VALUES(?, ?)", pote, 0)
    participantes = db.execute("SELECT * FROM participantes")
    transacoes = db.execute("SELECT * FROM transacoes")
    return render_template("poker.html", participantes=participantes, nro_participantes=nro_participantes, quantia_inicial=quantia_inicial, transacoes=transacoes)


In [11]:
@app.route("/poker2", methods=["POST", "GET"])
def poker2():
    nro_participantes = int(request.form.get("nro_part"))
    quantia_inicial = int(request.form.get("qt_inicial"))
    debito = request.form.get("debito")
    credito = request.form.get("credito")
    valor_s = request.form.get("valor")
    valor = int(valor_s)
    conta_cred = pd.read_sql("SELECT amount FROM participants WHERE id = {}".format(credito), conexao)
    conta_deb = pd.read_sql("SELECT amount FROM participants WHERE id = {}".format(debito), conexao)
    nome_cred = pd.read_sql("SELECT name_part FROM participants WHERE id = {}".format(credito), conexao)
    nome_deb = pd.read_sql("SELECT noname_part FROM participants WHERE id = {}".format(debito), conexao)
    saldo_cred = int(float(conta_cred[0]['amount'])) + valor
    pd.read_sql("UPDATE participants SET amount = ? WHERE id = {}".format(saldo_cred, credito), conexao)
    saldo_deb = int(float(conta_deb[0]['amount'])) - valor
    pd.read_sql("UPDATE participants SET amount = {} WHERE id = {}".format(saldo_deb, debito), conexao)
    pd.read_sql("INSERT INTO transactions(from_name, to_name, amount) VALUES({}, {}, {})".format(nome_deb[0]["nome"], nome_cred[0]["nome"], valor), conexao)
    participantes = pd.read_sql("SELECT * FROM participants", conexao)
    transacoes = pd.read_sql("SELECT * FROM transactions ORDER BY id DESC", conexao)
    return render_template("poker2.html", participantes=participantes, debito=debito, credito=credito, valor=valor, conta_deb=conta_deb, conta_cred=conta_cred, nro_participantes=nro_participantes, quantia_inicial=quantia_inicial, transacoes=transacoes )


## Código para SQL

CREATE TABLE transactions (
id INTEGER, 
from_name VARCHAR(45), 
to_name VARCHAR(45), 
amount INTEGER NOT NULL, 
PRIMARY KEY(id)
);

CREATE TABLE participants (
id INTEGER, 
name_part VARCHAR(45), 
amount INTEGER NOT NULL,
PRIMARY KEY(id));